<!-- dom:TITLE: Introduction to Python (MOD510): Classes -->
# Introduction to Python (MOD510): Classes
<!-- dom:AUTHOR: Oddbjørn Nødland -->
<!-- Author: -->  
**Oddbjørn Nødland**

Date: **Aug 20, 2019**

In [1]:
%matplotlib inline

import numpy as np
import math
import matplotlib.pyplot as plt

**Summary.** The aim of this workbook is to provide a rapid introduction to creating
your own Python classes.








## Creating custom classes

In addition to built-in object types and those shipped with
installable libraries, it is easy to create your own custom classes
in Python. Sometimes you will simply want to do this for code
reusability reasons.
In other situations, you may want to structure your entire code
base in an
[Object-Oriented-Programming (OOP)](https://en.wikipedia.org/wiki/Object-oriented_programming)
manner.

Consider the following piece of code:

In [2]:
class Person:

    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

The usage of the *class* statement tells Python that we wish to define a new
class. Here, our class only consists of two methods (class functions), both
of which are special in some way (they are reserved methods):

* *init* is the class constructor, which is called once a particular instance (object) of the class (object type, or 'blueprint') is created.

* *str* is used to provide a human readable string as output when applying a print statement to an object of the class.

Here is an example of how to use the class:

In [3]:
bill = Person('Bill')
steve = Person('Steve')
print(bill)
print(steve)

## Class variables versus instance variables

Notice that both class methods include *self* as the first argument.
This is Python's way for for an object to refer to itself, and to
distinguish between *class variables* and *instance variables*: While the
former are shared among all instances of a class, and therefore occupies
the same space in computer memory, the latter are different for each
particular object created of that class.

Class variables are assigned at the top level of a class statement, and
typically also right after the class statement, e.g.:

In [4]:
class Student:

    number_of_students = 0  # this is a class (shared) attribute

    def __init__(self, name):
        self.name = name  # this is an instance attribute
        Student.number_of_students += 1

    def __str__(self):
        return self.name

While the Person class had a single instance variable and no class variables,
the Student class has one of each. Notice that if we wish to change the
class variable inside a method, we can call it by first writing the class name.
Once we have created specific instances of the class (i.e., objects), we
can refer to class variables either via the individual objects, or again by
the class:

In [5]:
student1 = Student('Eric')
student2 = Student('John')
print(Student.number_of_students)
print(student1.number_of_students)

It is also possible to create class methods or static methods:

In [6]:
class Thing:

    number_of_things = 0
    info_messages_printed = 0

    def __init__(self, name):
        self.name_of_thing = name
        Thing.number_of_things += 1

    @classmethod
    def info(cls):  # for these methods we use cls instead of self
        print('There are {:d} things.'.format(Thing.number_of_things))
        # We can modify class attributes inside a class method, but not instance attributes:
        Thing.info_messages_printed += 1
        # Try to uncomment the next line:
        #self.name_of_thing = 'Unnamed'

    @staticmethod
    def do_nothing():
        # these functions can neither alter class nor object state.
        return

In [7]:
book = Thing('Book')
computer = Thing('Computer')
Thing.info()
Thing.do_nothing()

Instance variables are referenced via *self* (we could actually use a different
word than 'self', but it has become the accepted standard). *self* is also
used for a particular object to call its own methods (functions).
This is illustrated below, in a somewhat more interesting example:

In [8]:
class FluidFlowSimulator:

    def __init__(self, num_cells, length):

        # Numerical grid:
        self.num_cells = num_cells
        self.L = length
        self.dx = np.zeros(self.num_cells)
        self.dx.fill(self.L/self.num_cells)
        self.x_edges = np.linspace(0, self.L, self.num_cells+1)

        # Concentration data:
        self.fluid_concentrations = np.zeros(self.num_cells+1)

        # Initial & boundary conditions:
        self.c_init = 0.0
        self.c_inj = 1.0  # default

    def reset_to_initial_condition(self):

        self.fluid_concentrations.fill(self.c_init)

    def set_initial_and_boundary_condition(self, c_init, c_inj):

        self.c_init = c_init
        self.c_inj = c_inj
        self.fluid_concentrations.fill(self.c_init)

    def run(self, times):

        self.reset_to_initial_condition()  # call method
        global_timesteps = times[1:]-times[:-1]
        for dt in global_timesteps:
            self.advance_large_step(dt)  # call method

    def advance_large_step(self, dt):
        pass  # not implemented yet

The next code snippet illustrates how to use the class:

In [9]:
simulator = FluidFlowSimulator(10, 7.0)
simulator.set_initial_and_boundary_condition(10.0, 35.0)
t_end = 24.0*60*60  # one day in seconds
times = np.linspace(0, t_end, 49)
simulator.run(times)

## Operator overloading example

The next example shows how we might start if we want to represent complex
numbers with a custom class. We also include some more documentation:

In [10]:
# Example of how to create a new object type (custom class):
class ComplexNumber:

    def __init__(self, a, b):
        """
        Constructor. Initializes the complex number.

        :param a: Real part.
        :param b: Imaginary part.
        """

        self.real_part = a
        self.imaginary_part = b

    def __str__(self):
        """
       String representation of the complex number.

        :return:
        """
        return '{}+{}*i'.format(self.real_part, self.imaginary_part)

    def __add__(self, other):
        """
        Addition of complex numbers. Returns a new number.

        :param other: Complex number being added to this one.
        :return:
        """
        a = self.real_part + other.real_part
        b = self.imaginary_part + other.imaginary_part
        return ComplexNumber(a, b)

    def __sub__(self, other):
        """
        Subtraction of complex numbers. Returns a new number.

        :param other: Complex number being subtracted from this one.
        :return:
        """
        a = self.real_part - other.real_part
        b = self.imaginary_part - other.imaginary_part
        return ComplexNumber(a, b)

    def __mul__(self, other):
        """
        Multiplication of complex numbers. Returns a new number.

        :param other: Complex number being multiplied with this one.
        :return:
        """
        a = self.real_part*other.real_part - self.imaginary_part*other.imaginary_part
        b = self.real_part*other.imaginary_part + self.imaginary_part*other.real_part
        return ComplexNumber(a, b)

    def __truediv__(self, other):
        """
        Division of complex numbers. Returns a new number.

        :param other: Complex number with which the current one is divided.
        :return:
        """
        d = other.real_part**2 + other.imaginary_part**2
        if d == 0:
            raise(ZeroDivisionError('Cannot divide by zero.'))
        a = (self.real_part*other.real_part + self.imaginary_part*other.imaginary_part)/d
        b = (self.imaginary_part*other.real_part-self.real_part*other.imaginary_part)/d
        return ComplexNumber(a, b)

    def modulus(self):
        """
        :return: length (modulus) of the complex number in the complex plane.
        """
        return math.sqrt(self.real_part**2 + self.imaginary_part**2)

z1 = ComplexNumber(0, 1)
z2 = ComplexNumber(1, 0)
z3 = ComplexNumber(2, 0)
z4 = ComplexNumber(1, 1)

No attempt has been made to be efficient in this implementation. Also, there
is already a complex number class available in the
 [cmath](https://docs.python.org/2/library/cmath.html) module, which means
that you probably should use that instead of creating your own.
However, the above code illustrates several of the reasons why you might want
to work with classes. In particular, notice the usage of
[operator overloading](https://en.wikipedia.org/wiki/Operator_overloading),
which allow us to define arithmetic operators for complex numbers (addition,
subtraction, multiplication, and division):

In [11]:
# Examples of operator overloading (and usage of __str__ method):
print(z1+z2)
print(z1*z2)
print(z2/z3)

# Find length of z4 = 1+i in the complex plane:
print(z4.modulus())

## Callable objects

In numerical computing it can often be advantageous to represent mathematical
functions with a class, e.g.:

In [12]:
# Representing a function by a custom class:

class SineWaveFunction:
    """
    Class representation of a sine wave function:

        f(x) = A*sin(omega*t + phi)=A*sin(2*pi*f*t+phi)
    """

    def __init__(self, A=1.0, omega=1.0, phi=0.0):
        """
        Instantiate function. If given no input parameters,
        by default f(x)=sin(x).

        :param A: amplitude
        :param omega: angular frequency (=2*pi*f)
        :param phi: phase
        """

        self.A = A
        self.omega = omega
        self.phi = phi

        # Frequency from angular frequency:
        self.f = self.omega/(2.0*np.pi)

    def __call__(self, x):
        """
        Implements function call operator.

        Example of usage:

            f = SineWaveFunction()
            value = f(pi/2)
            print(value)  # <-- returns 1.0

        :param x: Input value to function.
        :return: Value of function at x.
        """
        return self.A*np.sin(self.omega*x+self.phi)

f = SineWaveFunction()
print(f(math.pi*0.5))

# Notice that we used the NumPy version of the sine function in __call__(x).
# This is to allow vectorized computation, e.g., as follows:
x_vals = np.linspace(0, 2*np.pi, 10)
print(f(x_vals))

This type of construct can be especially useful when we have to call the
function repeatedly as part of a larger computation, but where the function
needs to store an internal state that is prone to change in-between successive
calls. The reason why this works is the usage of yet another special (reserved)
method, namely *call*. Instances of classes which implement this method are
called *callable*, and having it allows us to treat class instances like
functions.

## Inheritance and polymorphism

Classes provide a convenient means by which to increase code reuse, and they
facilitate the storage of various types of information that 'naturally
belong together'. However, the reason for introducing them in the first
place was because of their more advanced features that support an
object-oriented-programming (OOP) style. We will not really delve
into OOP issues here, but we end with a simple example of how to use
class
[inheritance](https://www.python-course.eu/python3_inheritance.php) to achieve
[polymorphism](https://en.wikipedia.org/wiki/Polymorphism_(computer_science)):

In [13]:
# Example of achieving polymorphism via super- and subclasses (inheritance):
class Shape:

    def __init__(self):
        pass

    def area(self):
        pass

class Circle(Shape):

    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return np.pi*self.radius**2

class Square(Shape):

    def __init__(self, side):
        self.side_length = side

    def area(self):
        return self.side_length**2

class Rectangle(Shape):

    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length*self.width

shape1 = Circle(1.0)
shape2 = Square(1.0)
shape3 = Rectangle(1.0, 2.0)
shapes = [shape1, shape2, shape3]
for s in shapes:
    print(s.area())

We see that we have defined a super class (Shape) that has three subclasses,
each representing a particular geometric figure with its own way of computing
area. However, as seen from the outside they all behave the same way: We
can simply apply call the *area()* method to a Shape object, regardless of
whether it is actually a Circle, a Square, or a Rectangle!

OOP will not be a focus in this course, but it can still be very useful to
organize parts of your programs into classes. If nothing else, it can
increase code readability.